# 初始化

In [0]:
#@markdown - **挂载** 
from google.colab import drive
drive.mount('GoogleDrive')

In [0]:
#@markdown - **卸载**
!fusermount -u GoogleDrive

# 代码区

In [0]:
#@title 多层感知机 { display-mode: "both" }
# Multilayer Perceptron
# 该程序实现 TensorFlow 下的三层感知机对 mnist 的分类
import numpy as np
import os, sys, ctypes
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [0]:
#@markdown - **定义打印进度条函数**
def print_progress(progress, epoch_num, loss, acc):
    """
    This function draw an active progress bar.
    :param progress: Where we are:
                       type: float
                       value: [0,1]
    :param epoch_num: number of epochs for training
    :param loss: The loss for the specific batch in training phase.

    :return: Progressing bar
    """
    
    barLength = 30

    assert type(progress) is float, "id is not a float: %r" % id
    assert 0 <= progress <= 1, "variable should be between zero and one!"

    # 状态符号
    status = ""

    # 打印结束后返回 \r\n 换行
    if progress >= 1:
        progress = 1
        status = "\r\n"

    # 状态记录
    indicator = int(round(barLength*progress))

    list = [str(epoch_num), "#"*indicator , "-"*(barLength-indicator), progress*100, loss, acc, status]
    text = "\rEpoch {0[0]} {0[1]} {0[2]} {0[3]:.2f}% completed loss={0[4]:.3f}, acc={0[5]:.2f}%{0[6]}".format(list)
    sys.stdout.write(text)
    sys.stdout.flush()

In [0]:
#@markdown - **设置参数**
events_path = 'GoogleDrive/My Drive/Colab Notebooks/Tensorboard'
checkpoints_path = 'GoogleDrive/My Drive/Colab Notebooks/Checkpoints'
max_num_checkpoints = 3
num_class = 10
batch_size = 256 #@param {type: "integer"}
num_epochs = 2000 #@param {type: "integer"}
num_neurons = 256 #@param {type: "integer"}
num_batch = 100 # 每个 epoch 中包含的 mini-batch 训练的次数，应能被 num_epochs整除

initial_rate = 5e-3 #@param {type: "number"}
learning_rate_decay_factor = 0.95
decay_steps = 16.
k_p = 0.5 #@param {type: "number"}

online_test = True #@param {type: "boolean"}

In [0]:
#@markdown - **训练、测试用的图像与标签的提取**
mnist = input_data.read_data_sets("sample_data/MNIST_data", reshape=True, one_hot=True)
train_image = mnist.train.images
train_label = mnist.train.labels
test_image = mnist.test.images
test_label = mnist.test.labels

num_samples, num_features = train_image.shape

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting sample_data/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting sample_data/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting sample_data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting sample_data/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
#@markdown - **设置网络图**
graph = tf.Graph()
with graph.as_default():
    # learning rate 的选取策略：
    
    global_step = tf.Variable(0, name='global_step', trainable=False)
    decay_steps = decay_steps
    learning_rate = tf.train.exponential_decay(learning_rate=initial_rate,
                                                global_step=global_step,
                                                decay_steps=decay_steps,
                                                decay_rate=learning_rate_decay_factor,
                                                staircase=True,
                                                name='exponential_decay')
    
    # placeholder 的定义
    image_place = tf.placeholder(tf.float32, shape=[None, num_features], name='image')
    label_place = tf.placeholder(tf.float32, shape=[None, num_class], name='label')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    # 三层感知机的网络结构
    #-------------------fc-1--------------------
    output_fc1 = tf.contrib.layers.fully_connected(inputs=image_place, num_outputs=num_neurons, scope='fc-1')
    #-------------------fc-2--------------------
    output_fc2 = tf.contrib.layers.fully_connected(inputs=output_fc1, num_outputs=num_neurons, scope='fc-2')
    #-----------------dropout layer-------------
    output_dp = tf.contrib.layers.dropout(inputs=output_fc2, keep_prob=keep_prob, scope='dropout-layer')
    #-------------------fc-3--------------------
    output_pre_softmax = tf.contrib.layers.fully_connected(inputs=output_dp, num_outputs=num_class, scope='fc-3')

    # 定义 loss
    with tf.name_scope('loss'):
        loss_tensor = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output_pre_softmax,
                                                                            labels=label_place, name='loss_tensor'))

    # 定义 accuracy
    with tf.name_scope('accuracy'):
        prediction = tf.equal(tf.arg_max(output_pre_softmax, 1), tf.arg_max(label_place, 1))
        accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

    # 根据初值定义optimizer：(两种定义方式，第二种可以得到所有的 gradients 和 variables)

    # with tf.name_scope('train'):
    #     train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss_tensor, global_step=global_step)

    with tf.name_scope('train'):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        gradients_vars = optimizer.compute_gradients(loss_tensor)
        train_op = optimizer.apply_gradients(gradients_vars, global_step=global_step)
    
    # 定义 summaries
    tf.summary.scalar('loss', loss_tensor, collections=['train', 'test'])
    tf.summary.scalar('accuracy', accuracy, collections=['train', 'test'])
    tf.summary.scalar('global_step', global_step, collections=['train'])
    tf.summary.scalar('learning_rate', learning_rate, collections=['train'])

    # 按照 'train' 和 'test' 将两种不同类型的 summaries 分别 merge
    summary_train = tf.summary.merge_all('train')
    summary_test = tf.summary.merge_all('test')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use `tf.math.argmax` instead


In [0]:
max_acc = 99.0 # 高于此精度的模型将被saved
min_cross = 0.1
with tf.Session(graph=graph) as sess:
    # 定义 saver，初始化以及定义 train 和 test 的 summary writer
    saver = tf.train.Saver(max_to_keep=max_num_checkpoints)
    sess.run(tf.global_variables_initializer())

    train_summary_dir = os.path.join(events_path, 'summaries', 'train')
    train_summary_writer = tf.summary.FileWriter(train_summary_dir)
    train_summary_writer.add_graph(sess.graph)

    test_summary_dir = os.path.join(events_path, 'summaries', 'test')
    test_summary_writer = tf.summary.FileWriter(test_summary_dir)
    test_summary_writer.add_graph(sess.graph)

    # 随机梯度下降进行训练
    for num_epoch in range(num_epochs):

        image_batch, label_batch = mnist.train.next_batch(batch_size)
        batch_loss, batch_acc, _, batch_summ, num_train, lr = sess.run([loss_tensor, accuracy, train_op, summary_train, global_step, learning_rate],
                                                                    feed_dict={image_place: image_batch, label_place: label_batch,
                                                                                keep_prob: k_p})
        batch_acc *= 100
        progress = float(num_epoch % num_batch + 1) / num_batch
        num_epoch_batch = num_epoch // num_batch + 1
        print_progress(progress, num_epoch_batch, batch_loss, batch_acc)
        # print('Epoch '+str(num_epoch+1) + ', learning rate is %.4f' % lr + ', train accuracy is '+'{:.2f}%.'.format(batch_acc))
        # print('Epoch '+str(num_epoch+1) + ', learning rate is %.4f' % lr + ', train loss is '+ '{:.4f}, '.format(batch_loss) + \
        #         'accuracy is '+'{:.2f}%.'.format(batch_acc))
        train_summary_writer.add_summary(batch_summ, num_epoch)

        checkpoints_prefix = 'model.ckpt'
        if (num_epoch+1) % num_batch == 0:
            print('Learning rate is %.4f' % lr + ', train accuracy is '+'{:.2f}%.'.format(batch_acc))
            if (batch_loss <= min_cross) & (batch_acc > max_acc): # 按照要求保存网络模型
                min_cross = batch_loss
                max_acc = batch_acc
                saver.save(sess, os.path.join(checkpoints_path, checkpoints_prefix), global_step=num_epoch+1)
                # print("\033[0;31;40m\tModel restored ... ...\033[0m\n")
                print("Model restored ... ...\n")
                print('\n')

            if online_test: #定义是否实时显示测试集精度
                test_loss, test_acc, test_summ = sess.run([loss_tensor, accuracy, summary_test],
                                                            feed_dict={image_place: test_image, label_place: test_label,
                                                            keep_prob: 1.})
                test_acc *= 100
                print('Test accuracy is '+'{:.2f}%.\n'.format(test_acc))
                test_summary_writer.add_summary(test_summ, num_epoch)
    
    # 测试集精度
    test_accuracy = sess.run([accuracy], feed_dict={image_place: test_image, label_place: test_label, 
                                                    keep_prob: 1.})
    test_acc = test_accuracy[0]*100
    print("Final Test Accuracy is %.2f%%" % test_acc)

    train_summary_writer.close()
    test_summary_writer.close()
sess.close()

Epoch 1 ##############################  100.00% completed loss=0.185, acc=93.75%
Learning rate is 0.0037, train accuracy is 93.75%.
Test accuracy is 94.59%.

Epoch 2 ##############################  100.00% completed loss=0.094, acc=96.88%
Learning rate is 0.0027, train accuracy is 96.88%.
Test accuracy is 96.60%.

Epoch 3 ##############################  100.00% completed loss=0.094, acc=97.66%
Learning rate is 0.0020, train accuracy is 97.66%.
Test accuracy is 96.86%.

Epoch 4 ##############################  100.00% completed loss=0.027, acc=99.22%
Learning rate is 0.0015, train accuracy is 99.22%.
Model restored ... ...



Test accuracy is 97.36%.

Epoch 5 ##############################  100.00% completed loss=0.092, acc=96.09%
Learning rate is 0.0010, train accuracy is 96.09%.
Test accuracy is 97.66%.

Epoch 6 ##############################  100.00% completed loss=0.040, acc=98.83%
Learning rate is 0.0007, train accuracy is 98.83%.
Test accuracy is 97.68%.

Epoch 7 ##################